# Backpropagation handmade

In [1]:
# 1. Build the class for math expressions
# 2. Build some basic operations
# 3. Build computation graph
# 4. Build some activations
# 5. Build backpropagation

In [2]:
# inspired by (and 80% copied from :)) micrograd / Andrej Karpathy
# https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ

In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [4]:
class Parameter:
    def __init__(self, value: float, name: str, children=()) -> None:
        self._value = value
        self._name = name

        self._grad = 0.0
        self._backward = lambda: None

        self._prev = set(children)

    def __repr__(self) -> str:
        return f"Parameter {self._name} = {self._value}; dL/d[{self._name}] = {self._grad}"

    def __mul__(self, other):
        result = Parameter(
            self._value * other._value,
            f'{self._name} * {other._name}',
            (self, other)
        )

        def _backward():
            self._grad += other._value * result._grad #dL / dself
            other._grad += self._value * result._grad # dL / dother

        result._backward = _backward

        return result

    def __add__(self, other):
        result = Parameter(
            self._value + other._value,
            f'[{self._name} + {other._name}]',
            (self, other)
        )

        def _backward():
            self._grad += 1.0 * result._grad  #dL / dself
            other._grad += 1.0 * result._grad # dL / dother

        result._backward = _backward

        return result

    def sigmoid(self):
        # f(x) = 1 / (1 + exp(self._value))
        # f'(x) = f(x) * (1 - f(x))

        val = 1.0 / (1.0 + math.exp(-self._value))

        result = Parameter(
            val,
            f"σ({self._name})",
            (self, )
        )

        def _backward():
            self._grad = result._grad * val * (1 - val)

        result._backward = _backward

        return result
    
    def gelu(self):
        val = 0.5 * self._value * (1 + np.tanh(np.sqrt(2/np.pi) * (self._value + 0.044715 * self._value ** 3)))

        result = Parameter(
            val,
            f"gelu({self._name})",
            (self, )
        )

        def _backward():
            self._grad = result._grad * 0.5 * (1 + np.tanh(np.sqrt(2/np.pi) * (self._value + 0.044715 * np.power(self._value, 3)))) + 0.5 * self._value / np.power(np.cosh(np.sqrt(2/np.pi) * (self._value + 0.044715 * np.power(self._value, 3))), 2) * np.sqrt(2/np.pi) * (1 + 3 * 0.044715 * np.power(self._value, 2))

        result._backward = _backward

        return result

    def relu(self):
        val = np.maximum(0, self._value)

        result = Parameter(
            val,
            f"relu({self._name})",
            (self, )
        )

        def _backward():
            self._grad = result._grad * np.array(self._value >= 0).astype(np.float64)
        
        result._backward = _backward

        return result

    def backward(self):
       sorted = []
       visited = set()
       def topoligical_sort(v):
           if v not in visited:
               visited.add(v)
               for child in v._prev:
                   topoligical_sort(child)
               sorted.append(v)
       topoligical_sort(self)

       self._grad = 1.0
       for node in reversed(sorted):
           node._backward()

    

    
        
def sgd(weights: list, lr = 0.001):
    for param in weights:
        param._value -= param._grad * lr
    return weights

In [5]:
a = Parameter(9.0, 'a')
b = Parameter(2.0, 'b')
c = Parameter(5.0, 'c')
d = Parameter(5.0, 'd')

In [6]:
print(a)
print(b)
print(c)
print(d)

Parameter a = 9.0; dL/d[a] = 0.0
Parameter b = 2.0; dL/d[b] = 0.0
Parameter c = 5.0; dL/d[c] = 0.0
Parameter d = 5.0; dL/d[d] = 0.0


In [7]:
u = a * b
v = u + c
L = v * d

In [8]:
u, v, L

(Parameter a * b = 18.0; dL/d[a * b] = 0.0,
 Parameter [a * b + c] = 23.0; dL/d[[a * b + c]] = 0.0,
 Parameter [a * b + c] * d = 115.0; dL/d[[a * b + c] * d] = 0.0)

In [9]:
L.backward()

In [10]:
print(L)

Parameter [a * b + c] * d = 115.0; dL/d[[a * b + c] * d] = 1.0


In [11]:
print(v)
print(d)

Parameter [a * b + c] = 23.0; dL/d[[a * b + c]] = 5.0
Parameter d = 5.0; dL/d[d] = 23.0


In [12]:
print(a)
print(b)
print(c)
print(u)

Parameter a = 9.0; dL/d[a] = 10.0
Parameter b = 2.0; dL/d[b] = 45.0
Parameter c = 5.0; dL/d[c] = 5.0
Parameter a * b = 18.0; dL/d[a * b] = 5.0


In [13]:
x = Parameter(4.0, 'x')

f = x + x

f._grad = 1.0
f._backward()

In [14]:
print(f)
print(x)

Parameter [x + x] = 8.0; dL/d[[x + x]] = 1.0
Parameter x = 4.0; dL/d[x] = 2.0


In [51]:
x1 = Parameter(3.0, 'x1')
x2 = Parameter(4.0, 'x2')

w1 = Parameter(1.0, 'w1')
w2 = Parameter(2.0, 'w2')

x1w1 = x1 * w1
x2w2 = x2 * w2
xw = x1w1.relu() + x2w2.relu()
out = xw.gelu()


In [52]:
out.backward()

print(out)
print(xw)
print(x1w1)
print(x2w2)
print(w1)
print(w2)

Parameter gelu([relu(x1 * w1) + relu(x2 * w2)]) = 11.0; dL/d[gelu([relu(x1 * w1) + relu(x2 * w2)])] = 1.0
Parameter [relu(x1 * w1) + relu(x2 * w2)] = 11.0; dL/d[[relu(x1 * w1) + relu(x2 * w2)]] = 1.0
Parameter x1 * w1 = 3.0; dL/d[x1 * w1] = 1.0
Parameter x2 * w2 = 8.0; dL/d[x2 * w2] = 1.0
Parameter w1 = 1.0; dL/d[w1] = 3.0
Parameter w2 = 2.0; dL/d[w2] = 4.0


In [53]:
weights = sgd([w1, w2])
w1 = weights[0]
w2 = weights[1]

In [54]:
x1w1 = x1 * w1
x2w2 = x2 * w2
xw = x1w1.relu() + x2w2.relu()
out = xw.gelu()

In [55]:
out.backward()

print(out)
print(xw)
print(x1w1)
print(x2w2)
print(w1)
print(w2)

Parameter gelu([relu(x1 * w1) + relu(x2 * w2)]) = 10.975; dL/d[gelu([relu(x1 * w1) + relu(x2 * w2)])] = 1.0
Parameter [relu(x1 * w1) + relu(x2 * w2)] = 10.975; dL/d[[relu(x1 * w1) + relu(x2 * w2)]] = 1.0
Parameter x1 * w1 = 2.991; dL/d[x1 * w1] = 1.0
Parameter x2 * w2 = 7.984; dL/d[x2 * w2] = 1.0
Parameter w1 = 0.997; dL/d[w1] = 6.0
Parameter w2 = 1.996; dL/d[w2] = 8.0


In [57]:
for i in range(5):
    x1w1 = x1 * w1
    x2w2 = x2 * w2
    xw = x1w1.relu() + x2w2.relu()
    out = xw.gelu()
    print(out)
    out.backward()
    
    weights = sgd([w1, w2])
    w1 = weights[0]
    w2 = weights[1]

Parameter gelu([relu(x1 * w1) + relu(x2 * w2)]) = 10.35; dL/d[gelu([relu(x1 * w1) + relu(x2 * w2)])] = 0.0
Parameter gelu([relu(x1 * w1) + relu(x2 * w2)]) = 10.149999999999999; dL/d[gelu([relu(x1 * w1) + relu(x2 * w2)])] = 0.0
Parameter gelu([relu(x1 * w1) + relu(x2 * w2)]) = 9.924999999999999; dL/d[gelu([relu(x1 * w1) + relu(x2 * w2)])] = 0.0
Parameter gelu([relu(x1 * w1) + relu(x2 * w2)]) = 9.674999999999999; dL/d[gelu([relu(x1 * w1) + relu(x2 * w2)])] = 0.0
Parameter gelu([relu(x1 * w1) + relu(x2 * w2)]) = 9.399999999999999; dL/d[gelu([relu(x1 * w1) + relu(x2 * w2)])] = 0.0
